# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [11]:
%store -r aitsn_gdf gdf boundary_gdf project 
%store -r ndvi_paths ndvi_das ndvi_stack

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [12]:
# Import libraries
import matplotlib.pyplot as plt
import holoviews as hv

import json
import os
import pathlib

import pandas as pd
import geopandas as gpd
import hvplot.xarray
import hvplot.pandas

import rioxarray as rxr
import xarray as xr

from datetime import datetime
from glob import glob
import earthpy

# STEP 3: Plot NDVI

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Plot the change in NDVI spatially</div></div><div class="callout-body-container callout-body"><p>Complete the following:</p>
<ul>
<li>Select data from 2001 to 2011 (right before and after the water rights 
began to be restored in 2004) <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="1"></span>)</li>
<li>Take the temporal mean (over the <strong>date</strong>, not
spatially)</li>
<li>Get the NDVI variable (should be a DataArray, not a Dataset)</li>
<li>Repeat for the data from 2012 to 2022 (more recent) <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span>)</li>
<li>Subtract the 2001-2011 time period <strong>from</strong> the
2012-2022 time period</li>
<li>Plot the result using a <strong>diverging</strong> color map like
<code>cmap=plt.cm.PiYG</code></li>
</ul>
<p>There are different types of color maps for different types of data.
In this case, we want decreases to be a different color from increases,
so we should use a <strong>diverging</strong> color map. Check out
available colormaps in the <a
href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">matplotlib
documentation</a>.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?</div></div><div class="callout-body-container callout-body"><p>For an extra challenge, add the <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary to the plot.</p></div></div>

In [25]:
# Compute the difference in NDVI before and after

# Convert date object to datetime coordinate
ndvi_stack['date'] = xr.DataArray(ndvi_stack.date.values.astype('datetime64[ns]'), dims='date')

# Select data by date ranges
ndvi_0111 = ndvi_stack.sel(date=slice('2001-01-01', '2011-12-31'))
ndvi_1222 = ndvi_stack.sel(date=slice('2012-01-01', '2022-12-31'))
# Take temporal mean by date dimension
ndvi_0111_mean = ndvi_0111.mean(dim='date')
ndvi_1222_mean = ndvi_1222.mean(dim='date')
# Compute NDVI change through subtraction
ndvi_change = ndvi_1222_mean - ndvi_0111_mean

ndvi_change

<xarray.Dataset> Size: 315kB
Dimensions:      (x: 382, y: 203)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB -112.3 -112.3 -112.3 ... -111.5 -111.5 -111.5
  * y            (y) float64 2kB 33.39 33.39 33.38 33.38 ... 32.97 32.97 32.97
    spatial_ref  int64 8B 0
Data variables:
    NDVI         (y, x) float32 310kB -0.05568 -0.02921 ... -0.0209 -0.03258

In [19]:
# Plot the difference
ndvi_plot = ndvi_change.hvplot(
    geo=True,
    x='x',
    y='y',
    cmap=plt.cm.PiYG,
    colorbar=True,
    title='NDVI Change from 2001-2011 to 2012-2022',
    frame_width=600
)

boundary_plot = boundary_gdf.hvplot(
    geo=True,
    fill_color=None,
    line_color='black'
)
# Combine plots
ndvi_boundary_plot = (ndvi_plot * boundary_plot)

ndvi_boundary_plot

:Overlay
   .Image.I    :Image   [x,y]   (NDVI)
   .Polygons.I :Polygons   [Longitude,Latitude]

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [22]:
# Save datasets and plots too large for %store
ndvi_change.to_netcdf("ndvi_change.nc")

hv.save(ndvi_boundary_plot.opts(width=600), "ndvi_plot.html", fmt='html')

In [27]:
%store aitsn_gdf gdf boundary_gdf project 
%store ndvi_paths ndvi_das ndvi_stack

Stored 'aitsn_gdf' (GeoDataFrame)
Stored 'gdf' (GeoDataFrame)
Stored 'boundary_gdf' (GeoDataFrame)
Stored 'project' (Project)
Stored 'ndvi_paths' (list)
Stored 'ndvi_das' (list)
Stored 'ndvi_stack' (Dataset)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!